In [1]:
import torch 
import torch.nn as nn
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms

In [2]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = CIFAR10(root='./data', train=True, transform=transform_train, download=True)
testset = CIFAR10(root='./data', train=False, transform=transform_test, download=True) 

Files already downloaded and verified
Files already downloaded and verified


In [3]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle = True, num_workers = 2) 
testloader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle=False, num_workers=2)

In [4]:
from torch.nn import AvgPool2d
import torch.optim as optim

class LeNet5_v0(nn.Module):
    def __init__(self):
        torch.manual_seed(42)
        super().__init__()
        self.conv_layer = nn.Sequential(
            # c1, s2, c3, s4, c5, f6, output layer
            nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=0),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 120, kernel_size=5, stride=1, padding=0)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(120, 84),
            nn.Linear(84, 10)
        )

    def forward(self, X):
        x = self.conv_layer(X)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)
        return x


In [ ]:
model = LeNet5_v0()


loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
def accuracy_fn(y_true, y_pred):
    correct = (y_true == y_pred).sum().item()
    acc = (correct / len(y_true)) * 100
    return acc

def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device):

    model.train()
    train_loss, train_acc = 0.0, 0.0
    model.to(device)

    for batch, (X, y) in enumerate(data_loader):
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss and accuracy
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        train_acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))

        # 3. Zero gradients
        optimizer.zero_grad()

        # 4. Backward pass
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    train_loss /= len(data_loader)
    train_acc /= len(data_loader)

    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")
    return train_loss, train_acc


def test_step(model: torch.nn.Module,
              data_loader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device):

    model.eval()
    test_loss, test_acc = 0.0, 0.0
    model.to(device)

    with torch.inference_mode():
        for X, y in data_loader:
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            y_pred = model(X)

            # 2. Calculate loss and accuracy
            loss = loss_fn(y_pred, y)
            test_loss += loss.item()
            test_acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))

    test_loss /= len(data_loader)
    test_acc /= len(data_loader)

    print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%")
    return test_loss, test_acc

In [ ]:
EPOCHS = 10
for epoch in range(EPOCHS):
    train_loss, train_acc = train_step(model, trainloader, loss_fn, optimizer, accuracy_fn, device="cpu")
    test_loss, test_acc = test_step(model, testloader, loss_fn, accuracy_fn, device="cpu")

    print(f"Epoch [{epoch+1}/{EPOCHS}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% "
          f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")

/home/theyashwanthsai/.pyenv/versions/3.11.1/lib/python3.11/site-packages/torch/autograd/graph.py:768: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Train loss: 1.94225 | Train accuracy: 30.76%
Test loss: 1.84248 | Test accuracy: 36.21%
Epoch [1/10] Train Loss: 1.9422, Train Acc: 30.76% Test Loss: 1.8425, Test Acc: 36.21%
Train loss: 1.90711 | Train accuracy: 32.96%
Test loss: 1.81460 | Test accuracy: 37.04%
Epoch [2/10] Train Loss: 1.9071, Train Acc: 32.96% Test Loss: 1.8146, Test Acc: 37.04%
Train loss: 1.90179 | Train accuracy: 33.13%
Test loss: 1.81513 | Test accuracy: 36.91%
Epoch [3/10] Train Loss: 1.9018, Train Acc: 33.13% Test Loss: 1.8151, Test Acc: 36.91%
Train loss: 1.89610 | Train accuracy: 33.53%
Test loss: 1.80115 | Test accuracy: 37.76%
Epoch [4/10] Train Loss: 1.8961, Train Acc: 33.53% Test Loss: 1.8012, Test Acc: 37.76%
Train loss: 1.89413 | Train accuracy: 33.43%
Test loss: 1.79679 | Test accuracy: 37.56%
Epoch [5/10] Train Loss: 1.8941, Train Acc: 33.43% Test Loss: 1.7968, Test Acc: 37.56%
Train loss: 1.89268 | Train accuracy: 33.83%
Test loss: 1.81300 | Test accuracy: 37.07%
Epoch [6/10] Train Loss: 1.8927, Trai

In [ ]:
# Adding non linearity for better accuracy
# Original LeNet used tanh

class LeNet5_v1(nn.Module):
    def __init__(self):
        torch.manual_seed(42)
        super().__init__()
        self.conv_layer = nn.Sequential(
            # c1, s2, c3, s4, c5, f6, output layer
            nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),

            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(16, 120, kernel_size=5, stride=1, padding=0),
            nn.ReLU()
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10)
        )

    def forward(self, X):
        x = self.conv_layer(X)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)
        return x


In [15]:
model_1 = LeNet5_v1()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model_1.parameters(), lr = 0.001)

In [16]:
EPOCHS = 10
for epoch in range(EPOCHS):
    train_loss, train_acc = train_step(model_1, trainloader, loss_fn, optimizer, accuracy_fn, device="cpu")
    test_loss, test_acc = test_step(model_1, testloader, loss_fn, accuracy_fn, device="cpu")

    print(f"Epoch [{epoch+1}/{EPOCHS}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% "
          f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")

Epoch [1/10] Train Loss: 1.7566, Train Acc: 34.63% Test Loss: 1.4941, Test Acc: 45.18%
Epoch [2/10] Train Loss: 1.5067, Train Acc: 44.29% Test Loss: 1.3744, Test Acc: 50.06%
Epoch [3/10] Train Loss: 1.4062, Train Acc: 48.71% Test Loss: 1.2768, Test Acc: 54.12%
Epoch [4/10] Train Loss: 1.3385, Train Acc: 51.53% Test Loss: 1.2079, Test Acc: 56.51%
Epoch [5/10] Train Loss: 1.2853, Train Acc: 53.59% Test Loss: 1.1738, Test Acc: 58.41%
Epoch [6/10] Train Loss: 1.2509, Train Acc: 55.32% Test Loss: 1.1515, Test Acc: 58.28%
Epoch [7/10] Train Loss: 1.2099, Train Acc: 56.58% Test Loss: 1.1960, Test Acc: 58.25%
Epoch [8/10] Train Loss: 1.1858, Train Acc: 58.01% Test Loss: 1.0925, Test Acc: 61.00%
Epoch [9/10] Train Loss: 1.1592, Train Acc: 58.59% Test Loss: 1.0652, Test Acc: 62.67%
Epoch [10/10] Train Loss: 1.1341, Train Acc: 59.66% Test Loss: 1.0491, Test Acc: 62.70%
